
# Opportunity Atlas — Newark (NY) Replication & Extension (All Plotly)

**Author:** Zachary A. Smith, Ph.D. — Python port and extensions

This notebook reproduces the analysis from your senior seminar example and adds an *extension* to identify and map New York census tracts that outperform Newark (tract **36117021200**) for **Black children from families at the bottom 25th percentile** of the parental income distribution. All charts are implemented in **Plotly**.

**Data & Geography**
- Tract-level outcomes come from the **Opportunity Atlas** `atlas.dta` (2010 Census tract geographies; born 1978–83 cohorts). See the Atlas data description for variable definitions such as `kfr_black_p25` and subgroup counts.
- The Atlas is a collaboration between **Opportunity Insights** and the **U.S. Census Bureau** (Module 1: neighborhood outcomes; newer modules add county/metro trends and other outcomes).
- We map on **2010** New York Census tracts (TIGER/Line 2010), joining by **`GEOID10`** to align with `atlas.dta`.
- If you later want to show **2020** tracts, use Opportunity Insights’ **2010→2020 crosswalk** to carry estimates forward before mapping.

**Reliability note**
We set a conservative minimum subgroup size (default **`count_black ≥ 50`**) to avoid over-interpreting outcomes from tiny samples; this mirrors general small-cell privacy/reliability practices. You can adjust this threshold to show sensitivity.


In [ ]:

# %% [markdown]
# ## 0) Setup
# - Uses Plotly for all figures (no matplotlib).
# - GeoPandas is used once to convert shapefiles to GeoJSON (Option B).

import os, json, math
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

pd.options.display.float_format = '{:,.2f}'.format

TEMPLATE = 'plotly_white'
COLOR_SEQ = px.colors.qualitative.D3

def add_layout(fig, title, x=None, y=None):
    fig.update_layout(template=TEMPLATE, title=title, margin=dict(l=10,r=10,t=60,b=10),
                      legend=dict(orientation='h', y=1.08))
    if x: fig.update_xaxes(title=x)
    if y: fig.update_yaxes(title=y)
    return fig

def make_geoid(state, county, tract):
    return f"{int(state):02d}{int(county):03d}{int(tract):06d}"

print('Environment ready.')


In [ ]:

# %% [markdown]
# ## 1) Load Opportunity Atlas data (`atlas.dta`)
# Option A (default): read directly from your GitHub repo.
# Option B: place a local copy next to the notebook and switch the path.

GITHUB_RAW_URL = 'https://raw.githubusercontent.com/Prof-Smith/Opportunity_Atlas_Senior_Sem/main/atlas.dta'
LOCAL_ATLAS = 'atlas.dta'

try:
    atlas = pd.read_stata(GITHUB_RAW_URL)
    print('Loaded atlas.dta from GitHub.')
except Exception as e:
    print('Falling back to local atlas.dta due to:', e)
    atlas = pd.read_stata(LOCAL_ATLAS)
    print('Loaded local atlas.dta')

# Build 11-digit GEOID to match TIGER/Line 2010
atlas['geoid'] = atlas.apply(lambda r: make_geoid(r['state'], r['county'], r['tract']), axis=1)

# Key subsets
NY = atlas[atlas['state'] == 36].copy()                              # New York State
Wayne = atlas[(atlas['state'] == 36) & (atlas['county'] == 117)].copy()
Newark = atlas[(atlas['state'] == 36) & (atlas['county'] == 117) & (atlas['tract'] == 21200)].copy()
US = atlas.copy()

assert len(Newark) == 1, 'Expected exactly one row for Newark tract 36117021200.'
newark_row = Newark.iloc[0]
newark_geoid = newark_row['geoid']
newark_geoid



## 2) Convert uploaded TIGER/Line 2010 shapefile → GeoJSON (Option B)
This cell finds all `.shp` files under your cloned repo, converts them to **GeoJSON** (WGS84 / EPSG:4326), and writes them under `geojson/` while preserving subfolder structure. The 2010 TIGER layer provides **`GEOID10`** for joining to `atlas.dta`.


In [ ]:

# If running on Colab or a new environment, uncomment to install: 
# !pip install -q geopandas shapely fiona pyproj gitpython

from pathlib import Path
import subprocess

REPO_URL = 'https://github.com/Prof-Smith/Opportunity_Atlas_Senior_Sem.git'
REPO_DIR = Path('Opportunity_Atlas_Senior_Sem')
OUT_ROOT  = REPO_DIR / 'geojson'

# Clone if needed
if not REPO_DIR.exists():
    subprocess.run(['git', 'clone', REPO_URL, str(REPO_DIR)], check=True)

import geopandas as gpd
OUT_ROOT.mkdir(parents=True, exist_ok=True)

converted, errors = [], []
for shp_path in REPO_DIR.rglob('*.shp'):
    if 'geojson' in shp_path.parts:
        continue
    rel = shp_path.relative_to(REPO_DIR)
    base = rel.with_suffix('')
    out_dir = OUT_ROOT / base.parent
    out_dir.mkdir(parents=True, exist_ok=True)
    out_geo = out_dir / (base.name + '.geojson')
    try:
        print(f'Converting: {shp_path} -> {out_geo}')
        gdf = gpd.read_file(shp_path)
        # Reproject to WGS84 for web mapping
        try:
            if gdf.crs is None:
                gdf = gdf.set_crs(epsg=4269, allow_override=True).to_crs(epsg=4326)  # TIGER often NAD83
            elif gdf.crs.to_epsg() != 4326:
                gdf = gdf.to_crs(epsg=4326)
        except Exception as crs_err:
            print('  CRS handling issue:', crs_err)
        gdf.to_file(out_geo, driver='GeoJSON')
        converted.append((str(shp_path), str(out_geo)))
    except Exception as e:
        errors.append((str(shp_path), str(e)))
        print('  ERROR:', e)

print(f"Done. Converted {len(converted)} shapefile(s).")
if errors:
    print('Failures:')
    for shp, msg in errors:
        print(' -', shp, '->', msg)


In [ ]:

# Locate a 2010 NY tract GeoJSON (tl_2010_36_tract10*.geojson) and load it
from pathlib import Path

geojson_candidates = list((REPO_DIR / 'geojson').rglob('tl_2010_36_tract10*.geojson'))
if not geojson_candidates:
    print('No tl_2010_36_tract10 GeoJSON found under geojson/. Check conversion step.')
else:
    NY_TRACTS_GEOJSON = str(geojson_candidates[0])
    print('Using GeoJSON:', NY_TRACTS_GEOJSON)
    with open(NY_TRACTS_GEOJSON, 'r') as f:
        ny_geo = json.load(f)
    # Quick schema peek
    props = ny_geo['features'][0]['properties'] if ny_geo.get('features') else {}
    print('First feature properties (subset):', list(props.keys())[:10])



## 3) Newark (36117021200) — Table 1 replication (by race, P25)
We extract Newark’s race-specific outcomes for the P25 parent-income cohort and display a Plotly bar chart.


In [ ]:

def get_val(row, prefix, suffix):
    v = row.get(f'{prefix}_{suffix}', np.nan)
    try: return float(v)
    except: return np.nan

row = Newark.iloc[0]
rows = []
for label, rcode in [('White','white'), ('Black','black')]:
    rows.append({
        'Race': label,
        'Household Income @35 (P25)': get_val(row, 'kfr', f'{rcode}_p25'),
        'Individual Income (P25)':     get_val(row, 'kir', f'{rcode}_p25'),
        'Married @35 (P25)':           get_val(row, 'married', f'{rcode}_p25'),
        'Employment @35 (P25)':        get_val(row, 'emp', f'{rcode}_p25'),
        'Incarceration Rate (P25)':    get_val(row, 'ia', f'{rcode}_p25'),
        'Teen Birth Rate (P25)':       get_val(row, 'birrate', f'{rcode}_p25'),
        'Top 20% HH Income (P25)':     get_val(row, 'ktop20', f'{rcode}_p25'),
        'Top 20% Ind Income (P25)':    get_val(row, 'ktop20_ind', f'{rcode}_p25')
    })

table1_df = pd.DataFrame(rows)
print(table1_df)

# Plotly bar
t1 = table1_df.melt('Race', var_name='Outcome', value_name='Value').dropna()
fig = px.bar(t1, x='Outcome', y='Value', color='Race', barmode='group', color_discrete_sequence=COLOR_SEQ,
             title='Newark (36117021200) — Key Outcomes by Race (Parents @ P25)')
add_layout(fig, fig.layout.title.text).update_xaxes(tickangle=-30)
fig.show()



## 4) Newark vs New York vs US — P25 / P75 / P100 (weighted by subgroup counts)
We compute weighted means for NY and US using subgroup counts (`count_white`, `count_black`, `count_hisp`) and compare to Newark’s single-tract values.


In [ ]:

def weighted_mean(s, w):
    ok = (~s.isna()) & (~w.isna())
    return np.average(s[ok], weights=w[ok]) if ok.sum() else np.nan

def summarize(df, var, w):
    return weighted_mean(df[var], df[w])

targets = {
    '25th': {'white':'kfr_white_p25', 'hisp':'kfr_hisp_p25', 'black':'kfr_black_p25'},
    '75th': {'white':'kfr_white_p75', 'hisp':'kfr_hisp_p75', 'black':'kfr_black_p75'},
    '100th':{'white':'kfr_white_p100','hisp':'kfr_hisp_p100','black':'kfr_black_p100'}
}
weights = {'white':'count_white','hisp':'count_hisp','black':'count_black'}

rows = []
for pct in targets:
    for rcode, var in targets[pct].items():
        rows += [
            {'Group':'Newark','Percentile':pct,'Race':rcode.title(),'Household Income': float(Newark.iloc[0][var])},
            {'Group':'New York','Percentile':pct,'Race':rcode.title(),'Household Income': summarize(NY, var, weights[rcode])},
            {'Group':'US','Percentile':pct,'Race':rcode.title(),'Household Income': summarize(US, var, weights[rcode])},
        ]
summary_df = pd.DataFrame(rows)
summary_df.head()

for pct in ['25th','75th','100th']:
    tmp = summary_df[summary_df['Percentile']==pct]
    fig = px.bar(tmp, x='Race', y='Household Income', color='Group', barmode='group',
                 color_discrete_sequence=COLOR_SEQ,
                 title=f"Children’s HH Income (Parents @ {pct}) — Newark vs NY vs US")
    add_layout(fig, fig.layout.title.text, 'Race', 'Household income ($)')
    fig.show()



## 5) Scatter + trendline — P25 vs P75 (Wayne County)
We visualize the P25–P75 relationship for pooled data and by race for Wayne County (Plotly scatter + OLS line).


In [ ]:

def scatter_fit(df, x, y, title, xlab, ylab):
    d = df[[x,y]].dropna()
    if d.empty:
        print('No data available for', x, y)
        return None
    b1, b0 = np.polyfit(d[x], d[y], 1)
    line_x = np.linspace(d[x].min(), d[x].max(), 100)
    line_y = b1*line_x + b0
    fig = px.scatter(d, x=x, y=y, template=TEMPLATE)
    fig.add_trace(go.Scatter(x=line_x, y=line_y, mode='lines', name=f'OLS: y={b1:.3f}x+{b0:.1f}', line=dict(color='black')))
    return add_layout(fig, title, xlab, ylab)

fig = scatter_fit(Wayne, 'kfr_pooled_p25','kfr_pooled_p75',
                  'Wayne County — P25 vs P75 (Pooled)',
                  'P25 HH income ($)','P75 HH income ($)')
if fig: fig.show()

for rcode, rlabel in [('white','White'),('black','Black'),('hisp','Hispanic')]:
    x, y = f'kfr_{rcode}_p25', f'kfr_{rcode}_p75'
    fig = scatter_fit(Wayne, x, y, f'Wayne County — P25 vs P75 ({rlabel})',
                      'P25 HH income ($)','P75 HH income ($)')
    if fig: fig.show()



## 6) Extension — Identify NY “Opportunity Zones” (Black @ P25) vs Newark
**Definition:** A NY tract is an opportunity zone if `kfr_black_p25` exceeds Newark’s `kfr_black_p25`. We apply a conservative filter, default **`count_black ≥ 50`**, to avoid highlighting very small Black child populations.


In [ ]:

# Thresholds 
min_black_children = 50
newark_thresh = float(newark_row['kfr_black_p25'])

nyb = NY[NY['count_black'] >= min_black_children].copy()
nyb['delta_vs_newark'] = nyb['kfr_black_p25'] - newark_thresh
nyb['is_opzone'] = nyb['delta_vs_newark'] > 0

num_zones = int(nyb['is_opzone'].sum())
print('Opportunity zones (NY, Black @ P25, count_black >=', min_black_children, '):', num_zones)

# Top 10 tracts above Newark
top10 = nyb.sort_values('delta_vs_newark', ascending=False).head(10)[
    ['geoid','county','tract','kfr_black_p25','count_black','delta_vs_newark']
].reset_index(drop=True)
top10

fig_top = px.bar(top10.assign(label=lambda d: d['geoid']),
                 x='label', y='delta_vs_newark',
                 title=f'Top 10 NY Tracts Above Newark — Black @ P25 (n={num_zones})',
                 labels={'label':'Tract GEOID','delta_vs_newark':'Excess over Newark ($)'},
                 color_discrete_sequence=['#2ca02c'])
add_layout(fig_top, fig_top.layout.title.text).update_xaxes(tickangle=-45)
fig_top.show()



## 7) Geographic visualizations (Plotly choropleths)
We join Atlas rows to the **2010 NY tracts GeoJSON** by **`GEOID10`** and render: (i) a continuous delta map and (ii) a binary highlight of tracts that outperform Newark. If your GeoJSON uses a different property name, change `featureidkey` accordingly. TIGER/Line 2010 typically exposes **`GEOID10`**.


In [ ]:

# Prepare choropleth frame
chorodf = nyb[['geoid','kfr_black_p25','count_black','delta_vs_newark','is_opzone']].copy()

if 'ny_geo' not in globals():
    raise RuntimeError('GeoJSON not loaded. Re-run the conversion/loading cells above.')

fig_map = px.choropleth_mapbox(
    chorodf, geojson=ny_geo, locations='geoid', featureidkey='properties.GEOID10',
    color='delta_vs_newark',
    color_continuous_scale=['#440154','#3b528b','#21908d','#5ec962','#fde725'],
    mapbox_style='open-street-map', opacity=0.75,
    center={'lat': 42.95, 'lon': -75.5}, zoom=5,
    hover_data={'geoid':True,'kfr_black_p25':':,.0f','count_black':True,'delta_vs_newark':':,.0f','is_opzone':True},
    title='Δ vs Newark (Black @ P25), New York Tracts — 2010 geographies'
)
add_layout(fig_map, fig_map.layout.title.text)
fig_map.show()

fig_bin = px.choropleth_mapbox(
    chorodf.assign(Status=lambda d: np.where(d['is_opzone'], 'Outperformed Newark', 'Did not outperform')),
    geojson=ny_geo, locations='geoid', featureidkey='properties.GEOID10',
    color='Status', color_discrete_sequence=['#2ca02c','#d62728'],
    mapbox_style='open-street-map', opacity=0.80,
    zoom=5, center={'lat':42.95,'lon':-75.5},
    title=f'“Opportunity Zones” (Black @ P25): {num_zones:,} of {len(chorodf):,} NY tracts (count_black ≥ {min_black_children})'
)
add_layout(fig_bin, fig_bin.layout.title.text)
fig_bin.show()



## 8) Covariates (Wayne County) — Plotly correlations
Employment (2000), Poverty (1990), and District test scores (3rd grade, 2013) vs P25 outcomes.


In [ ]:

for col, label in [('emp2000','Employment Rate, 2000'),
                   ('poor_share1990','Poverty Rate, 1990'),
                   ('gsmn_math_g3_2013','Std. Test Score (3rd grade, 2013)')]:
    if col in Wayne.columns:
        d = Wayne[[col, 'kfr_pooled_p25']].dropna().rename(columns={col:'x','kfr_pooled_p25':'y'})
        if not d.empty:
            b1, b0 = np.polyfit(d['x'], d['y'], 1)
            r = d.corr().iloc[0,1]
            fig = px.scatter(d, x='x', y='y', title=f'P25 HH Income vs {label} — r={r:.2f}',
                             labels={'x':label,'y':'HH income @ P25 ($)'})
            fig.add_trace(go.Scatter(x=np.linspace(d['x'].min(), d['x'].max(), 100),
                                     y=b1*np.linspace(d['x'].min(), d['x'].max(), 100)+b0,
                                     mode='lines', name='OLS fit', line=dict(color='black')))
            add_layout(fig, fig.layout.title.text, label, 'HH income @ P25 ($)')
            fig.show()



## 9) Save selected outputs
CSV exports for class use.


In [ ]:

summary_df.to_csv('ny_newark_us_summary_by_race.csv', index=False)

# top10 from earlier cell
try:
    top10.to_csv('ny_opportunity_zones_top10_black_p25.csv', index=False)
    print('Saved CSVs to working directory.')
except NameError:
    print('Top10 frame not found. Run the extension cell first.')



---
### Notes for instructors
- The Atlas file is built on **2010** tracts; keep geographies consistent when joining to shapefiles. If using 2020 tracts, apply the **2010→2020** crosswalk first.
- Consider discussing small-sample reliability with students and running sensitivity checks using `min_black_children` = 25, 50, 100.
